<a href="https://colab.research.google.com/github/Penitto/risk_project1/blob/master/Iriski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Выделенные риск-факторы

1. Фондовый риск — риск снижения цены акций; __индекс РТС и МОЕКС__
2. Процентный риск — риск изменения процентных ставок; __процентные ставки__
3. Валютный риск — риск изменения курсов валют; __курс доллара и курс юаня__
4. Товарный риск — риск изменения цен товаров; __цены на нефть__


# Стохастическая модель динамики 

In [1]:
import numpy as np
import pandas as pd
import os
from scipy.interpolate import CubicSpline

In [263]:
class Portfolio():
    def __init__(self, bonds, shares, cur, risk):
        self.bonds = bonds
        self.shares = shares
        self.cur = cur
        self.risk = risk
        self.price = 0
        
    # Посчитать цену портфеля в определённый день
    def countPriceInDate(self, date):
        res = 0
        
        for i in self.bonds:
            res += i[3] * i[1]['<CLOSE>'].loc[date]
            
        for i in self.shares:
            res += i[3] * i[1]['<CLOSE>'].loc[date]
            
        
    # Посчитать объём портфеля в уе
    def countInitialValue(self):
        for i in range(len(self.bonds)):
            amount = self.bonds[i][2] / self.bonds[i][1]['<CLOSE>'][0]
            self.bonds[i] = (*self.bonds[i], amount)
            
        for i in range(len(self.shares)):
            amount = self.shares[i][2] / self.shares[i][1]['<CLOSE>'][0]
            self.shares[i] = (*self.shares[i], amount)
            
        for i in range(len(self.cur)):
            amount = self.cur[i][2] / self.cur[i][1]['<CLOSE>'][0]
            self.cur[i] = (*self.cur[i], amount)

In [2]:
# Загрузка данных в класс
shares = ['./shares/AFLT_160101_200101.csv', 
          './shares/GAZP_160101_200101.csv',
          './shares/GMKN_160101_200101.csv', 
          './shares/KMAZ_160101_200101.csv', 
          './shares/LKOH_160101_200101.csv', 
          './shares/PIKK_160101_200101.csv', 
          './shares/MGNT_160101_200101.csv', 
          './shares/RBCM_160101_200101.csv', 
          './shares/ROSN_160101_200101.csv', 
          './shares/SBER_160101_200101.csv']

shares_name = [i[9:13] for i in shares]

# Нужно интерполировать
bonds = ['./bonds/SU26212RMFS9_160101_200101.csv',
         './bonds/SU26205RMFS3_160101_200101.csv',
         './bonds/SU26207RMFS9_160101_200101.csv', 
         './bonds/SU26209RMFS5_160101_200101.csv', 
         './bonds/SU26211RMFS1_160101_200101.csv']

bonds_name = [i[8:20] for i in bonds]

currencies = ['./index/USD_RUB.csv', './index/CNY_RUB.csv']

currencies_name = [i[8:15] for i in currencies]

indexes = ['./index/ICE.BRN_160101_200101.csv', 
           './index/IMOEX_160101_200101.csv', 
           './index/RTSI_160101_200101.csv']

indexes_name = ['Brent', 'MOEX', 'RTSI']

zero_bond = './zerobond.csv'

days = 1010

In [3]:
risk_df = pd.read_csv(shares[0], index_col='<DATE>').drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1)
risk_df.index = pd.to_datetime(risk_df.index)
k = 1
for i in shares[1:]:
    tmp = pd.read_csv(i, index_col='<DATE>') \
            .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
            .rename(columns={'<CLOSE>': shares_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1
    
k = 0
for i in bonds:
    tmp = pd.read_csv(i, index_col='<DATE>') \
            .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
            .rename(columns={'<CLOSE>' : bonds_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1
    
k = 0
for i in currencies:
    tmp = pd.read_csv(i, index_col='Date') \
            .drop(['Open', 'High', 'Low', 'Change %'], axis=1) \
            .rename(columns={'Price' : currencies_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1
    
k = 0
for i in indexes:
    tmp = pd.read_csv(i, index_col='<DATE>') \
            .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
            .rename(columns={'<CLOSE>' : indexes_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1

risk_df = risk_df.rename(columns={'<CLOSE>' : shares_name[0]})

zero_bond_df = pd.read_csv(zero_bond, sep=';', index_col='Date')
zero_bond_df.index = pd.to_datetime(zero_bond_df.index)

risk_df = risk_df.join(zero_bond_df, how='left')
risk_df = risk_df.fillna(risk_df.mean(axis=0))

In [44]:
tmp = risk_df.diff()
temp = risk_df.iloc[:-1,:]
temp.index = tmp.iloc[1:,:].index
tmp = tmp.iloc[1:,:] / temp

,AFLT,GAZP,GMKN,KMAZ,LKOH,PIKK,MGNT,RBCM,ROSN,SBER,...,9,12,24,36,60,84,120,180,240,360
<DATE>,,,,,,,,,,,,,,,,,,,,,
2016-05-01,-0.031139,0.011415,0.018269,-0.001385,0.030214,-0.010673,-0.001408,0.007400,0.008765,0.008979,...,-0.010309,-0.010091,-0.006951,-0.005015,-0.006122,-0.006205,-0.001052,0.003195,0.005348,0.005353
2016-06-01,-0.007346,-0.003738,-0.010457,0.002774,-0.000840,-0.003283,-0.002643,-0.012592,-0.003160,-0.020629,...,0.001042,0.003058,0.009000,0.010081,0.011294,0.013528,0.011579,0.008493,0.006383,0.005325
2016-11-01,-0.036818,-0.033544,-0.043382,-0.015214,-0.050505,-0.016941,-0.057686,-0.023379,-0.044968,-0.060093,...,0.097815,0.081301,0.042616,0.039920,0.050761,0.053388,0.046826,0.047368,0.049683,0.050847
2016-12-01,0.022858,0.016441,-0.005465,-0.005618,-0.015235,0.005744,-0.028124,-0.011970,-0.004771,0.007470,...,0.006635,0.009398,0.012357,0.011516,0.010628,0.012671,0.017893,0.019095,0.018127,0.018145
2016-01-13,0.032864,-0.016774,0.007132,0.011299,0.000675,0.004760,-0.001447,0.002203,-0.001251,-0.002290,...,-0.007533,-0.011173,-0.008451,0.000949,0.005736,0.001925,0.000000,0.000000,0.000989,0.001980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,-0.009608,-0.000510,-0.002338,-0.005254,-0.000247,0.001014,0.009431,-0.011425,0.000887,-0.000523,...,-0.007299,-0.005435,-0.003509,-0.003413,-0.004886,-0.006299,-0.006116,-0.005970,-0.004431,-0.001464
2019-12-25,0.002522,-0.006356,0.013245,0.008803,0.000082,-0.001519,0.013199,0.064477,0.005760,-0.002533,...,-0.009191,-0.010929,-0.005282,-0.005137,-0.001637,0.000000,-0.001538,0.000000,0.000000,0.000000
2019-12-26,0.005806,0.003356,-0.033685,0.010471,0.000658,0.009381,0.009660,-0.015429,-0.003304,0.000806,...,0.001855,0.005525,0.003540,-0.001721,-0.008197,-0.006339,-0.001541,0.000000,0.001484,0.001466


In [56]:
def get_data():
    #Здесь - подгрузка и подготовка данных

    shares = ['./shares/AFLT_160101_200101.csv', 
              './shares/GAZP_160101_200101.csv',
              './shares/GMKN_160101_200101.csv', 
              './shares/KMAZ_160101_200101.csv', 
              './shares/LKOH_160101_200101.csv', 
              './shares/PIKK_160101_200101.csv', 
              './shares/MGNT_160101_200101.csv', 
              './shares/RBCM_160101_200101.csv', 
              './shares/ROSN_160101_200101.csv', 
              './shares/SBER_160101_200101.csv']

    shares_name = [i[9:13] for i in shares]

    # Нужно интерполировать
    bonds = ['./bonds/SU26212RMFS9_160101_200101.csv',
             './bonds/SU26205RMFS3_160101_200101.csv',
             './bonds/SU26207RMFS9_160101_200101.csv', 
             './bonds/SU26209RMFS5_160101_200101.csv', 
             './bonds/SU26211RMFS1_160101_200101.csv']

    bonds_name = [i[8:20] for i in bonds]

    currencies = ['./index/USD_RUB.csv', './index/CNY_RUB.csv']

    currencies_name = [i[8:15] for i in currencies]

    indexes = ['./index/IMOEX_160101_200101.csv', 
               './index/RTSI_160101_200101.csv', 
               './index/ICE.BRN_160101_200101.csv']

    indexes_name = ['MOEX', 'RTSI', 'Brent']

    zero_bond = './zerobond.csv'

    act_instruments = pd.read_csv(shares[0], index_col='<DATE>').drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1)
    act_instruments.index = pd.to_datetime(act_instruments.index)
    k = 1
    for i in shares[1:]:
        tmp = pd.read_csv(i, index_col='<DATE>') \
                .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
                .rename(columns={'<CLOSE>': shares_name[k]})
        tmp.index = pd.to_datetime(tmp.index)
        act_instruments = act_instruments.join(tmp, how='left')
        k += 1
        
    k = 0
    for i in bonds:
        tmp = pd.read_csv(i, index_col='<DATE>') \
                .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
                .rename(columns={'<CLOSE>' : bonds_name[k]})
        tmp.index = pd.to_datetime(tmp.index)
        act_instruments = act_instruments.join(tmp, how='left')
        k += 1

    act_instruments = act_instruments.rename(columns={'<CLOSE>' : shares_name[0]})
    act_instruments = act_instruments.fillna(act_instruments.mean(axis=0))
        
    act_risks = pd.read_csv(indexes[0], index_col='<DATE>').drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1)
    act_risks.index = pd.to_datetime(act_risks.index) 

    k = 1
    for i in indexes[1:]:
        tmp = pd.read_csv(i, index_col='<DATE>') \
                .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
                .rename(columns={'<CLOSE>' : indexes_name[k]})
        tmp.index = pd.to_datetime(tmp.index)
        act_risks = act_risks.join(tmp, how='left')
        k += 1

    k = 0
    for i in currencies:
        tmp = pd.read_csv(i, index_col='Date') \
                .drop(['Open', 'High', 'Low', 'Change %'], axis=1) \
                .rename(columns={'Price' : currencies_name[k]})
        tmp.index = pd.to_datetime(tmp.index)
        act_risks = act_risks.join(tmp, how='left')
        k += 1
    
    zero_bond_df = pd.read_csv(zero_bond, sep=';', index_col='Date')
    zero_bond_df.index = pd.to_datetime(zero_bond_df.index)
    act_risks = act_risks.join(zero_bond_df, how='left')
    act_risks = act_risks.fillna(act_risks.mean(axis=0))

    tmp = act_risks.diff()
    temp = act_risks.iloc[:-1,:]
    temp.index = tmp.iloc[1:,:].index
    r_risks = tmp.iloc[1:,:] / temp
    
    tmp = act_instruments.diff()
    temp = act_instruments.iloc[:-1,:]
    temp.index = tmp.iloc[1:,:].index
    r_instruments = tmp.iloc[1:,:] / temp

    # act_risks = pd.DataFrame() # реальные значения риск-факторов
    # act_instruments = pd.DataFrame() # реальные значения инструментов
    # r_risks = pd.DataFrame() # Риск-факторы, посчитанные в арифметических процентах
    # r_instruments = pd.DataFrame() # доходность инструментов, посчитанная в арифметических процентах

    return    r_risks, r_instruments, act_risks, act_instruments

get_data()

(             <CLOSE>      RTSI     Brent   USD_RUB   CNY_RUB         3  \
 <DATE>                                                                   
 2016-05-01  0.011282  0.004564 -0.019823 -0.064637 -0.041805 -0.015964   
 2016-06-01 -0.003170 -0.021097 -0.060399  0.061448  0.020197  0.002317   
 2016-11-01 -0.037762 -0.051152 -0.080861 -0.056343 -0.082938  0.117919   
 2016-12-01  0.004612 -0.005450 -0.024367  0.010346 -0.007364  0.003102   
 2016-01-13 -0.003266 -0.002402 -0.019137  0.198458  0.250925 -0.003093   
 ...              ...       ...       ...       ...       ...       ...   
 2019-12-24 -0.001061  0.003557  0.009311 -0.004836 -0.004099 -0.011009   
 2019-12-25  0.000082  0.002376 -0.122757 -0.001904  0.000690 -0.012987   
 2019-12-26  0.000274 -0.006003  0.133151  0.005900  0.004871 -0.001880   
 2019-12-27  0.006201  0.009480  0.000000 -0.002903 -0.002789 -0.062147   
 2019-12-30 -0.001508 -0.000310 -0.002095 -0.000756  0.000530 -0.038153   
 
                    6  

In [9]:
risk_df

,AFLT,GAZP,GMKN,KMAZ,LKOH,PIKK,MGNT,RBCM,ROSN,SBER,...,9,12,24,36,60,84,120,180,240,360
<DATE>,,,,,,,,,,,,,,,,,,,,,
2016-04-01,56.20,134.91,8922.0,36.10,2310.2,215.5,11366.0,4.730,251.00,98.01,...,9.70,9.91,10.07,9.97,9.80,9.67,9.51,9.39,9.35,9.34
2016-05-01,54.45,136.45,9085.0,36.05,2380.0,213.2,11350.0,4.765,253.20,98.89,...,9.60,9.81,10.00,9.92,9.74,9.61,9.50,9.42,9.40,9.39
2016-06-01,54.05,135.94,8990.0,36.15,2378.0,212.5,11320.0,4.705,252.40,96.85,...,9.61,9.84,10.09,10.02,9.85,9.74,9.61,9.50,9.46,9.44
2016-11-01,52.06,131.38,8600.0,35.60,2257.9,208.9,10667.0,4.595,241.05,91.03,...,10.55,10.64,10.52,10.42,10.35,10.26,10.06,9.95,9.93,9.92
2016-12-01,53.25,133.54,8553.0,35.40,2223.5,210.1,10367.0,4.540,239.90,91.71,...,10.62,10.74,10.65,10.54,10.46,10.39,10.24,10.14,10.11,10.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,103.08,254.87,19630.0,56.80,6080.5,395.0,3371.5,3.288,451.40,248.67,...,5.44,5.49,5.68,5.84,6.11,6.31,6.50,6.66,6.74,6.82
2019-12-25,103.34,253.25,19890.0,57.30,6081.0,394.4,3416.0,3.500,454.00,248.04,...,5.39,5.43,5.65,5.81,6.10,6.31,6.49,6.66,6.74,6.82
2019-12-26,103.94,254.10,19220.0,57.90,6085.0,398.1,3449.0,3.446,452.50,248.24,...,5.40,5.46,5.67,5.80,6.05,6.27,6.48,6.66,6.75,6.83


In [10]:
risk_df.diff()

,AFLT,GAZP,GMKN,KMAZ,LKOH,PIKK,MGNT,RBCM,ROSN,SBER,...,9,12,24,36,60,84,120,180,240,360
<DATE>,,,,,,,,,,,,,,,,,,,,,
2016-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-01,-1.75,1.54,163.0,-0.05,69.8,-2.3,-16.0,0.035,2.20,0.88,...,-0.10,-0.10,-0.07,-0.05,-0.06,-0.06,-0.01,0.03,0.05,0.05
2016-06-01,-0.40,-0.51,-95.0,0.10,-2.0,-0.7,-30.0,-0.060,-0.80,-2.04,...,0.01,0.03,0.09,0.10,0.11,0.13,0.11,0.08,0.06,0.05
2016-11-01,-1.99,-4.56,-390.0,-0.55,-120.1,-3.6,-653.0,-0.110,-11.35,-5.82,...,0.94,0.80,0.43,0.40,0.50,0.52,0.45,0.45,0.47,0.48
2016-12-01,1.19,2.16,-47.0,-0.20,-34.4,1.2,-300.0,-0.055,-1.15,0.68,...,0.07,0.10,0.13,0.12,0.11,0.13,0.18,0.19,0.18,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,-1.00,-0.13,-46.0,-0.30,-1.5,0.4,31.5,-0.038,0.40,-0.13,...,-0.04,-0.03,-0.02,-0.02,-0.03,-0.04,-0.04,-0.04,-0.03,-0.01
2019-12-25,0.26,-1.62,260.0,0.50,0.5,-0.6,44.5,0.212,2.60,-0.63,...,-0.05,-0.06,-0.03,-0.03,-0.01,0.00,-0.01,0.00,0.00,0.00
2019-12-26,0.60,0.85,-670.0,0.60,4.0,3.7,33.0,-0.054,-1.50,0.20,...,0.01,0.03,0.02,-0.01,-0.05,-0.04,-0.01,0.00,0.01,0.01


In [322]:
t = 1
vol = [(-1 + np.sqrt(1 + (1 / 1009) * np.sum((log_risk_df[log_risk_df.columns[i]] - t*np.mean(log_risk_df[log_risk_df.columns[i]]))**2))) / (t/2) for i in range(len(log_risk_df.columns))]
vol

[0.0003363680786643286,
 0.00022263484914519793,
 0.0002478627077189266,
 0.00023063651043608857,
 0.0002082510551275618,
 0.00015922427297976327,
 0.00033199229841684996,
 0.0006864559612309584,
 0.00021325533449401135,
 0.00031425224759162873,
 1.9707334205421034e-05,
 1.892297248096142e-05,
 1.648825343281146e-05,
 3.2760371751816564e-05,
 4.054345030013451e-05,
 0.0017398221114346413,
 0.0015505512608657845,
 0.0017711011586829528,
 8.302295768203294e-05,
 0.0001942056649917312,
 0.00037679647883415157,
 0.00032179392122255734,
 0.00027969926071991225,
 0.00024819053719138395,
 0.00018720182886244885,
 0.00016848211794906476,
 0.0001463856283714371,
 0.00012789560697124003,
 9.992366073019454e-05,
 7.474625599446227e-05,
 6.726635424714189e-05,
 7.480160537909342e-05]

In [11]:
def get_index():
    moex_ind = pd.read_csv('./index/IMOEX_160101_200101.csv')
    rts_ind = pd.read_csv('./index/RTSI_160101_200101.csv')
    zerobond = pd.read_csv('./zerobond.csv', sep=';')
    
    weeks_in_month = (365/7)*(1/12)
    maturity_rub = [3,6,9,12,24,36,60,84,120,180,240,360,]
    maturity_rub= [i * weeks_in_month for i in maturity_rub]
    
    interpolate_rub = CubicSpline(maturity_rub, ds_rate_rub)
    
    interval_rub = np.arange(0, 54, 2)
    
    df_rub_int = pd.DataFrame(data={'maturity_rub_2weeks': interval_rub, 'rub_act': interpolate_rub(interval_rub)})
    
    df_rub_usd_int=pd.concat([df_usd_int, df_rub_int], axis=1, sort=False)
    df_rub_usd_int=df_rub_usd_int.drop(['maturity_rub_2weeks'], axis=1)
    df_rub_usd_int=df_rub_usd_int.rename(columns={"maturity_usd_2weeks":"maturity"})
    df_rub_usd_int['maturity_frac'] = df_rub_usd_int['maturity']/54

    # s=0.0134
    s=1/0.01442

    new_rates = pd.concat(
        [
            df_rub_usd_int,
            df_rub_usd_int.diff(1).rename(columns={x:x.replace('act','diff') for x in df_rub_usd_int.columns})
        ],
        axis=1)

    new_rates.fillna(0, inplace=True)
    new_rates['fx_act']=(s*(1+new_rates['usd_act']*0.01)/(1+new_rates['rub_act']*0.01))
    new_rates['fx_diff'] =new_rates['fx_act'].diff()


    curve_rub_act = new_rates.loc[1:,'rub_act']
    curve_usd_act = new_rates.loc[1:,'usd_act']
    curve_fx_act = new_rates.loc[1:,'fx_act']

    curve_rub_diff = new_rates.loc[1:,'rub_diff']
    curve_usd_diff = new_rates.loc[1:,'usd_diff']
    curve_fx_diff = new_rates.loc[1:,'fx_diff']
    init = new_rates.loc[0,['rub_act','usd_act','fx_act']]


    return  (
        curve_rub_act,
        curve_usd_act,
        curve_fx_act,
        curve_rub_diff,
        curve_usd_diff,
        curve_fx_diff,
        init)
    
    
    
    
    def stoch_wrapper(decomp):
        def make_stoch(num):
            sigma=[0.03, 0.0093, 0.11]
            stoch_generator = np.dot(np.random.normal(size=(num,3)),decomp)*sigma
            return stoch_generator
        return make_stoch

    
stoch_generator = stoch_wrapper(get_decomp())

def simulate_hull_white(
    sim_number = 10,):
    rub_alpha=0.03
    sigma=[0.03, 0.0093, 0.11]
    k_fx=0.015
    dt=14/365
    timesteps = 26

    (
        curve_rub,
        curve_rub_df,
        init
        ) = get_rates()


    results = np.zeros(shape=(timesteps+1, 3, sim_number))

    passed_time=0

    for sim_ix in range(sim_number):
        results[0, :, sim_ix] = init
        stochs = stoch_generator(timesteps+1)
        for i, (rate_rub, df_rub, stoch_tuple) in enumerate(zip(curve_rub,curve_rub_df, stochs)):
            
            passed_time += dt

            theta_rub = df_rub + rub_alpha * rate_rub + (sigma[0]**2) * (1 - np.exp(-2 * rub_alpha * passed_time)) / 2 * rub_alpha

            results[i + 1, 0, sim_ix] = (theta_rub - rub_alpha* results[:, 0, sim_ix].sum()) * dt + stoch_tuple[0]

    return results

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,RTSI,D,04/01/16,00:00:00,755.16,761.68,741.67,749.28,204806463
1,RTSI,D,05/01/16,00:00:00,747.94,753.97,743.42,752.70,242929972
2,RTSI,D,06/01/16,00:00:00,753.06,754.04,735.62,736.82,210962101
3,RTSI,D,11/01/16,00:00:00,721.68,721.68,699.13,699.13,0
4,RTSI,D,12/01/16,00:00:00,689.33,706.31,681.46,695.32,0


# Оценка справедливой стоимости в зависимости от риск-факторов

# Оценка риска по портфелю

# Простая количественная валидация